<a href="https://colab.research.google.com/github/nalgo-intern/team-a/blob/master/sample_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 8.8MB/s 


In [0]:
downloaded = drive.CreateFile({'id':'1UAlxDtZompZuoqEI7hfdN47gGgA6RW5E'})

downloaded.GetContentFile('train.zip')

In [0]:
downloaded = drive.CreateFile({'id':'1xhHhaH92EIfwZ0lNGO_VFLiWQL7qvBV7'})

downloaded.GetContentFile('validation.zip')

In [4]:
!unzip train.zip
!unzip validation.zip

Archive:  train.zip
   creating: train/chimpanzee/
  inflating: train/chimpanzee/000001.jpg  
  inflating: train/chimpanzee/000002.jpg  
  inflating: train/chimpanzee/000003.jpg  
  inflating: train/chimpanzee/000006.jpg  
  inflating: train/chimpanzee/000007.jpg  
  inflating: train/chimpanzee/000009.jpg  
  inflating: train/chimpanzee/000010.jpg  
  inflating: train/chimpanzee/000011.jpg  
  inflating: train/chimpanzee/000012.jpg  
  inflating: train/chimpanzee/000013.jpg  
  inflating: train/chimpanzee/000014.jpg  
  inflating: train/chimpanzee/000015.jpg  
  inflating: train/chimpanzee/000016.jpg  
  inflating: train/chimpanzee/000017.jpg  
  inflating: train/chimpanzee/000019.jpg  
  inflating: train/chimpanzee/000020.jpg  
  inflating: train/chimpanzee/000021.jpg  
  inflating: train/chimpanzee/000022.jpg  
  inflating: train/chimpanzee/000023.png  
  inflating: train/chimpanzee/000024.jpg  
  inflating: train/chimpanzee/000025.JPG  
  inflating: train/chimpanzee/000027.jpg  
  i

In [6]:
from keras.models import Model
from keras.layers import Dense,Input,GlobalMaxPooling2D,Dropout
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

N_CATEGORIES = 3
IMAGE_SIZE = 224
BATCH_SIZE = 8

NUM_TRAINING = 288
NUM_VALIDATION = 72

input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model = VGG16(weights='imagenet', include_top=False,input_tensor=input_tensor)

x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(.25)(x)
x = Dense(1024, activation='relu')(x)


predictions = Dense(N_CATEGORIES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers[:15]:
   layer.trainable = False

model.compile(optimizer=SGD(lr=1e-4, momentum=0.9), loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

json_string=model.to_json()
open("model"+'.json','w').write(json_string)

train_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
   shear_range=0,
   zoom_range=0.1,
   horizontal_flip=True,
   rotation_range=0)

test_datagen = ImageDataGenerator(
   rescale=1.0 / 255,
)
train_generator = train_datagen.flow_from_directory(
   'train',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

validation_generator = test_datagen.flow_from_directory(
   'validation',
   target_size=(IMAGE_SIZE, IMAGE_SIZE),
   batch_size=BATCH_SIZE,
   class_mode='categorical',
   shuffle=True
)

hist = model.fit_generator(train_generator,
   steps_per_epoch=NUM_TRAINING//BATCH_SIZE,
   epochs=40,
   verbose=1,
   validation_data=validation_generator,
   validation_steps=NUM_VALIDATION//BATCH_SIZE,
   )

model.save('monkey.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
upload_file_2 = drive.CreateFile()
upload_file_2.SetContentFile("monkey.hdf5")
upload_file_2.Upload()

In [0]:
from google.colab import files
files.download('model.json')

In [0]:
downloaded = drive.CreateFile({'id':'1ThJebD-LznN8Rf0_699uJEnXKilUSa9e'})

downloaded.GetContentFile('test.zip')

In [10]:
!unzip test.zip

Archive:  test.zip
  inflating: test/img_563079ee59e6ee71a36fa5dbedecedc6126476.jpg  
